# Domain-Specific Fine-Tuning

### This notebook fine tunes the model and returns the result after training.

Let's first install the libraries we will use during this step:

In [1]:
pip install torch transformers python-dotenv langchain_pinecone datasets pinecone-client langchain-community scikit-learn langchain datasets transformers==4.17

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install wandb --upgrade

     ---------------------------------------- 0.0/67.9 kB ? eta -:--:--
     ------ --------------------------------- 10.2/67.9 kB ? eta -:--:--
     ----------------- -------------------- 30.7/67.9 kB 262.6 kB/s eta 0:00:01
     ---------------------------------- --- 61.4/67.9 kB 469.7 kB/s eta 0:00:01
     -------------------------------------- 67.9/67.9 kB 369.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   -- ------------------------------------- 0.2/3.8 MB 5.1 MB/s eta 0:00:01
   ---------------- ----------------------- 1.6/3.8 MB 20.0 MB/s eta 0:00:01
   ---------------------------------------  3.8/3.8 MB 30.5 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 24.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/897.5 kB ? eta -:--:--
   --------------------------------------- 897.5/897.5 kB 28.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
optimum 1.19.2 requires transformers[sentencepiece]<4.41.0,>=4.26.0, but you have transformers 4.17.0 which is incompatible.
sentence-transformers 2.6.1 requires transformers<5.0.0,>=4.32.0, but you have transformers 4.17.0 which is incompatible.


In [9]:
pip install --upgrade transformers

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install accelerate -U


  Using cached accelerate-0.30.1-py3-none-any.whl.metadata (18 kB)
   ---------------------------------------- 0.0/302.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/302.6 kB ? eta -:--:--
   -- ------------------------------------ 20.5/302.6 kB 165.2 kB/s eta 0:00:02
   ----- --------------------------------- 41.0/302.6 kB 281.8 kB/s eta 0:00:01
   ------------------ ------------------- 143.4/302.6 kB 774.0 kB/s eta 0:00:01
   ---------------------------------------- 302.6/302.6 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.2
    Uninstalling accelerate-0.29.2:
      Successfully uninstalled accelerate-0.29.2


# Setup

In [ ]:
# Python version
import sys 
print(sys.version)

3.12.0 (tags/v3.12.0:0fb18b0, Oct  2 2023, 13:03:39) [MSC v.1935 64 bit (AMD64)]


In [ ]:
# Environment Variables
from dotenv import load_dotenv
import yaml
import os
env_path = '/notebooks/TFM_LAW_LLM/env.txt'

# Load env
load_dotenv(dotenv_path=env_path)

False


We need the following torch configuration:

In [ ]:
# Torch config
from torch import cuda, bfloat16, float16
import torch

# Torch options
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# Parameters

In [ ]:
# Load parameters from YAML file
import os

# Change the current working directory to the directory containing the YAML file
os.chdir('/notebooks/TFM_LAW_LLM')

# Load parameters from YAML file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [ ]:
# Use optimum
use_optimum = config["use_optimum"]

# Show
use_optimum

False

# References

- https://github.com/the-ogre/LLM-FinetuningBERTforQuestionAnswering
- https://heidloff.net/article/fine-tuning-question-answering/
- https://medium.com/@xiaohan_63326/fine-tune-fine-tuning-bert-for-question-answering-qa-task-5c29e3d518f1

# Directory

In [ ]:
# Set directory to file location
from pathlib import Path
notebook_location = Path(os.path.abspath(''))
os.chdir(notebook_location)

# Get the current working directory
current_directory = os.getcwd()
current_directory

'C:\\Users\\polri\\Desktop\\Git_TFM\\TFM_LAW_LLM'

# Libraries

In [10]:
# General utility libraries
import os 

# PyTorch library for deep learning
import torch  

# HuggingFace Transformers library for natural language processing
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer 

# Dataset handling library from HuggingFace
from datasets import Dataset  

# Pinecone related
import pinecone  
from pinecone import Pinecone  

# LangChain integration with Pinecone
from langchain_pinecone import PineconeVectorStore  

# Transformers pipeline 
from transformers import pipeline  

# LangChain library for creating embeddings with HuggingFace
from langchain.embeddings.huggingface import HuggingFaceEmbeddings  

# Scikit-learn library for machine learning
from sklearn.model_selection import train_test_split  

# Local custom functions
from functions import *
from utils import *


## Embeddings

We load the embedding model.

In [ ]:
embed_model_id = config["embed_model_id"]
embed_model = HuggingFaceEmbeddings(model_name=embed_model_id, model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'})

c:\Users\polri\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
401 Client Error. (Request ID: Root=1-665df042-64c49e2e19465a9c026af3a0)

Repository Not Found for url: https://huggingface.co/dariolopez/roberta-base-bne-finetuned-msmarco-qa-es-mnrl-mn/resolve/main/model.safetensors.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
User Access Token "nacho" is expired


OSError: dariolopez/roberta-base-bne-finetuned-msmarco-qa-es-mnrl-mn is not a local folder or a valid repository name on 'https://hf.co'.

# Data

Creation of a synthetic dataset with, questions, answers and contexts to train the model.

In [ ]:
data = {
    "question": [
        "¿Cuál es la ley que regula el derecho al honor y la intimidad?",
        "¿Qué artículo de la Constitución Española establece la igualdad ante la ley?",
        "¿Qué establece el artículo 15 de la Constitución Española?",
        "¿Qué es el habeas corpus según la ley española?",
        "¿Qué regula la Ley Orgánica 3/2007?",
        "¿Qué establece el artículo 47 de la Constitución Española?",
        "¿Cuál es el objetivo de la Ley de Enjuiciamiento Criminal?",
        "¿Qué regula el artículo 20 de la Constitución Española?",
        "¿Qué es el Tribunal Constitucional en España?",
        "¿Cuál es el propósito de la Ley de Propiedad Intelectual?",
        "¿Qué establece el artículo 18 de la Constitución Española?",
        "¿Qué es la Ley de Transparencia?",
        "¿Qué regula el artículo 21 de la Constitución Española?",
        "¿Qué es el derecho de asilo según la ley española?",
        "¿Qué establece la Ley de Protección de Datos Personales?",
        "¿Qué es el recurso de amparo según la Constitución Española?",
        "¿Qué establece el artículo 27 de la Constitución Española?",
        "¿Qué regula la Ley General de la Seguridad Social?",
        "¿Qué es el Tribunal Supremo en España?",
        "¿Qué regula la Ley de Arrendamientos Urbanos?",
        "¿Qué tipo de acciones u omisiones no serán castigadas?",
        "¿Quién debe dictar la sentencia firme para ejecutar la pena o medida de seguridad?",
        "¿En qué se fundamentan las medidas de seguridad según el artículo 6?",
        "¿A qué tipos de delitos y faltas se aplicarán las disposiciones de este Título?",
        "¿Qué son según el artículo 10?",
        "¿Qué define como delitos graves el artículo 13?",
        "¿Quiénes no serán responsables criminalmente según el artículo 19?",
        "¿Qué condición exime de responsabilidad criminal según el artículo 20?",
        "¿Qué constituye la tentativa según el artículo 16?",
        "¿Qué es la provocacion según el artículo 18?",
        "¿Qué establece el artículo 25 de la Constitución Española?",
        "¿Qué define el artículo 19 del Código Penal sobre la responsabilidad criminal de los menores?",
        "¿Qué regula el artículo 23 del Código Penal?",
        "¿Qué es el principio de legalidad según el Código Penal?",
        "¿Qué establece el artículo 33 del Código Penal sobre las penas?",
        "¿Qué regula el artículo 50 del Código Penal?",
        "¿Qué se entiende por delito de lesa humanidad según el Código Penal?",
        "¿Cuál es la función de las penas según el artículo 25 del Código Penal?",
        "¿Qué establece el artículo 44 del Código Penal sobre la suspensión de la ejecución de la pena?",
        "¿Qué es la reincidencia según el Código Penal?",
        "¿Qué regula el artículo 76 del Código Penal sobre el concurso de delitos?",
        "¿Qué establece el artículo 55 del Código Penal sobre la responsabilidad civil derivada del delito?",
        "¿Qué es la eximente de legítima defensa según el Código Penal?",
        "¿Qué regula el artículo 88 del Código Penal sobre la sustitución de las penas?",
        "¿Qué establece el artículo 21 del Código Penal sobre las circunstancias atenuantes?",
        "¿Qué se entiende por delito continuado según el Código Penal?",
        "¿Qué regula el artículo 83 del Código Penal sobre la libertad condicional?",
        "¿Qué establece el artículo 61 del Código Penal sobre la expulsión de extranjeros?",
        "¿Qué es el cumplimiento sucesivo de penas según el Código Penal?",
        "¿Qué regula el artículo 100 del Código Penal sobre la extinción de la responsabilidad criminal?",
        "¿Qué se entiende por tentativa inacabada según el Código Penal?",
        "¿Qué regula el artículo 70 del Código Penal sobre la prescripción de delitos?",
        "¿Qué establece el artículo 56 del Código Penal sobre la prohibición de acercamiento a la víctima?",
        "¿Qué se entiende por concurso medial según el Código Penal?",
        "¿Qué regula el artículo 101 del Código Penal sobre la reincidencia de delitos graves?",
        "¿Qué establece el artículo 90 del Código Penal sobre las medidas de seguridad?",
        "¿Qué es la peligrosidad criminal según el Código Penal?",
        "¿Qué regula el artículo 45 del Código Penal sobre las costas procesales?",
        "¿Qué establece el artículo 35 del Código Penal sobre las penas privativas de libertad?",
        "¿Qué se entiende por omisión del deber de socorro según el Código Penal?",
        "¿Qué regula el artículo 31 del Código Penal sobre la responsabilidad penal de las personas jurídicas?",
        "¿Qué establece el artículo 57 del Código Penal sobre la inhabilitación absoluta?",
        "¿Qué define el artículo 62 del Código Penal sobre la prescripción de las penas?",
        "¿Qué es el delito de cohecho según el Código Penal?",
        "¿Qué regula el artículo 138 del Código Penal sobre el homicidio?",
        "¿Qué establece el artículo 181 del Código Penal sobre los abusos sexuales?",
        "¿Qué es el delito de falsedad documental según el Código Penal?",
        "¿Qué regula el artículo 237 del Código Penal sobre el robo?",
        "¿Qué establece el artículo 248 del Código Penal sobre la estafa?",
        "¿Qué es el delito de blanqueo de capitales según el Código Penal?",
        "¿Qué regula el artículo 301 del Código Penal sobre el tráfico de drogas?",
        "¿Qué establece el artículo 317 del Código Penal sobre los delitos contra los derechos de los trabajadores?",
        "¿Qué es la asociación ilícita según el Código Penal?",
        "¿Qué regula el artículo 367 del Código Penal sobre el contrabando?",
        "¿Qué establece el artículo 391 del Código Penal sobre la quiebra fraudulenta?",
        "¿Qué es el delito de acoso sexual según el Código Penal?",
        "¿Qué regula el artículo 400 del Código Penal sobre la insolvencia punible?",
        "¿Qué establece el artículo 419 del Código Penal sobre el tráfico de influencias?",
        "¿Qué es el delito de terrorismo según el Código Penal?",
        "¿Qué regula el artículo 510 del Código Penal sobre los delitos de odio?",
        "¿Qué establece el artículo 551 del Código Penal sobre el atentado contra la autoridad?",
        "¿Qué es el delito de injurias según el Código Penal?",
        "¿Qué regula el artículo 623 del Código Penal sobre los delitos leves contra el patrimonio?",
        "¿Qué establece el artículo 639 del Código Penal sobre la usurpación de funciones públicas?",
        "¿Qué es la omisión del deber de socorro según el Código Penal?",
        "¿Qué regula el artículo 637 del Código Penal sobre la alteración del orden público?",
        "¿Qué establece el artículo 641 del Código Penal sobre la tenencia ilícita de armas?",
        "¿Qué es el delito de tráfico de órganos según el Código Penal?",
        "¿Qué regula el artículo 643 del Código Penal sobre los delitos contra la salud pública?",
        "¿Qué establece el artículo 667 del Código Penal sobre la responsabilidad penal del menor?"
    ],
    "context": [
        "La Ley Orgánica 1/1982 regula el derecho al honor, a la intimidad personal y familiar y a la propia imagen.",
        "El artículo 14 de la Constitución Española establece que los españoles son iguales ante la ley.",
        "El artículo 15 de la Constitución Española establece el derecho a la vida y a la integridad física y moral.",
        "El habeas corpus es un procedimiento que protege la libertad individual contra arrestos y detenciones arbitrarias.",
        "La Ley Orgánica 3/2007 regula la igualdad efectiva entre mujeres y hombres.",
        "El artículo 47 de la Constitución Española establece el derecho a disfrutar de una vivienda digna y adecuada.",
        "La Ley de Enjuiciamiento Criminal regula el proceso penal en España.",
        "El artículo 20 de la Constitución Española regula la libertad de expresión y de información.",
        "El Tribunal Constitucional es el intérprete supremo de la Constitución Española.",
        "La Ley de Propiedad Intelectual protege los derechos de los autores sobre sus obras.",
        "El artículo 18 de la Constitución Española regula el derecho a la intimidad personal y familiar y a la inviolabilidad del domicilio.",
        "La Ley de Transparencia promueve la transparencia de la actividad pública y el derecho de acceso a la información.",
        "El artículo 21 de la Constitución Española regula el derecho de reunión pacífica y sin armas.",
        "El derecho de asilo protege a las personas que buscan refugio en España por persecución en sus países de origen.",
        "La Ley de Protección de Datos Personales regula el tratamiento de datos personales y la protección de los derechos de los individuos.",
        "El recurso de amparo es un mecanismo constitucional que protege los derechos fundamentales.",
        "El artículo 27 de la Constitución Española establece el derecho a la educación.",
        "La Ley General de la Seguridad Social regula la protección social en España.",
        "El Tribunal Supremo es el órgano jurisdiccional superior en todos los órdenes en España.",
        "La Ley de Arrendamientos Urbanos regula los contratos de alquiler de viviendas y locales.",
        "TITULO PRELIMINAR\nDe las garantias penales y de la aplicacion de la Ley penal\nArticulo 1.\n1. No sera castigada ninguna accion ni omision que\nno este prevista como delito o falta por Ley anterior\na su perpetracion.",
        "TITULO PRELIMINAR\nDe las garantias penales y de la aplicacion de la Ley penal\nArticulo 3.\n1. No podra ejecutarse pena ni medida de seguridad\nsino en virtud de sentencia firme dictada por el Juez\no Tribunal competente, de acuerdo con las leyes procesales.",
        "TITULO PRELIMINAR\nDe las garantias penales y de la aplicacion de la Ley penal\nArticulo 6.\n1. Las medidas de seguridad se fundamentan en\nla peligrosidad criminal del sujeto al que se impongaiı,\nexteriorizada en la comision de un hecho previsto como\ndelito.",
        "TITULO PRELIMINAR\nDe las garantias penales y de la aplicacion de la Ley penal\nArticulo 9.\nLas disposiciones de este Título se aplicaran a los\ndelitos y faltas que se hallen penados por leyes especiales.",
        "TITULO I\nDe la infraccion penal\nCAPITULO I\nDe los delitos y faltas\nArticulo 10.\n.son delitos o faltas las acciones y omisiones dolosas\no imprudentes penadas por la Ley.",
        "TITULO I\nDe la infraccion penal\nCAPITULO I\nDe los delitos y faltas\nArticulo 13.\n1. Son delitos graves las infracciones que la Ley\ncastiga con pena grave.",
        "TITULO I\nDe la infraccion penal\nCAPITULO II\nDe las causas que eximen de la responsabilidad\ncriminal\nArticulo 19.\nLos menores de dieciocho anos no seran responsables criminalmente con arreglo a este C.6digo.",
        "TITULO I\nDe la infraccion penal\nCAPITULO II\nDe las causas que eximen de la responsabilidad\ncriminal\nArticulo 20.\n1.° El que al tiempo de cometer la infraccion penal,\na causa de cualquier anomalia o alteracion psiquica, no\npueda comprender la ilicitud del hecho o actuar conforme\na esa comprension.",
        "TITULO I\nDe la infraccion penal\nCAPITULO II\nDe las causas que eximen de la responsabilidad\ncriminal\nArticulo 16.\n1. Hay tentativa cuando el sujeto da principio a la\n ejecucion del delito directamente por hechos exteriores,\npracticando todos o parte de los actos que objetivamente\ndeberian producir el resultado, y sin embargo este no\nse produce por causas independientes de la voluntad\ndel autor.",
        "TITULO I\nDe la infraccion penal\nCAPITULO II\nDe las causas que eximen de la responsabilidad\ncriminal\nArticulo 18.\n1. La provocacion existe cuando directamente se\nincita por medio de la imprenta, la radiodifusion o cualquier\notro medio de eficacia semejante, que facilite la publicidad,\na la perpetracion de un delito.",
        "El artículo 25 de la Constitución Española establece que las penas privativas de libertad y las medidas de seguridad estarán orientadas hacia la reeducación y reinserción social.",
        "El artículo 19 del Código Penal establece que los menores de dieciocho años no serán responsables criminalmente con arreglo a este Código.",
        "El artículo 23 del Código Penal regula la extradición y el cumplimiento de penas impuestas por tribunales extranjeros.",
        "El principio de legalidad, según el Código Penal, establece que no puede imponerse pena alguna sin previa ley que la establezca.",
        "El artículo 33 del Código Penal establece las diferentes clases de penas, clasificándolas en graves, menos graves y leves.",
        "El artículo 50 del Código Penal regula la multa como pena, incluyendo su cuantía y los criterios para su determinación.",
        "El delito de lesa humanidad, según el Código Penal, comprende actos inhumanos cometidos como parte de un ataque generalizado o sistemático contra una población civil.",
        "El artículo 25 del Código Penal establece que las penas tienen como función principal la prevención de la comisión de nuevos delitos.",
        "El artículo 44 del Código Penal regula la suspensión de la ejecución de la pena en ciertos casos y bajo determinadas condiciones.",
        "La reincidencia, según el Código Penal, se entiende como la comisión de un nuevo delito por una persona que ya ha sido condenada anteriormente por un delito de la misma naturaleza.",
        "El artículo 76 del Código Penal regula el concurso de delitos, estableciendo cómo se deben computar las penas cuando una persona es condenada por varios delitos.",
        "El artículo 55 del Código Penal establece la responsabilidad civil derivada del delito, incluyendo la obligación de reparación del daño causado.",
        "La eximente de legítima defensa, según el Código Penal, permite a una persona eximirse de responsabilidad criminal si actúa en defensa propia o de terceros frente a una agresión ilegítima.",
        "El artículo 88 del Código Penal regula la sustitución de las penas privativas de libertad por otras penas menos gravosas en ciertos casos.",
        "El artículo 21 del Código Penal establece las circunstancias atenuantes que pueden reducir la gravedad de la pena impuesta.",
        "El delito continuado, según el Código Penal, se entiende como la realización de varias acciones u omisiones que infringen el mismo precepto legal y responden a un plan preconcebido.",
        "El artículo 83 del Código Penal regula la concesión de la libertad condicional a los penados que cumplan determinados requisitos.",
        "El artículo 61 del Código Penal establece las condiciones bajo las cuales se puede expulsar a un extranjero del territorio español como consecuencia de una condena penal.",
        "El cumplimiento sucesivo de penas, según el Código Penal, se refiere a la ejecución de varias penas de privación de libertad una tras otra cuando una persona ha sido condenada por múltiples delitos.",
        "El artículo 100 del Código Penal regula las causas de extinción de la responsabilidad criminal, como el cumplimiento de la pena, la amnistía, y el indulto.",
        "La tentativa inacabada, según el Código Penal, ocurre cuando una persona inicia la ejecución de un delito pero no completa todos los actos necesarios para su consumación.",
        "El artículo 70 del Código Penal regula la prescripción de delitos, estableciendo los plazos después de los cuales no puede perseguirse un delito.",
        "El artículo 56 del Código Penal establece la prohibición de acercamiento a la víctima como medida de protección en casos de violencia o amenazas.",
        "El concurso medial, según el Código Penal, se entiende como la comisión de varios delitos mediante una sola acción que cumple todos los elementos de cada delito.",
        "El artículo 101 del Código Penal regula la reincidencia en delitos graves, estableciendo condiciones más severas para quienes reinciden en la comisión de estos delitos.",
        "El artículo 90 del Código Penal regula las medidas de seguridad que se pueden imponer a los delincuentes peligrosos, además de las penas privativas de libertad.",
        "La peligrosidad criminal, según el Código Penal, se refiere a la probabilidad de que una persona cometa futuros delitos basándose en su conducta pasada y otras circunstancias.",
        "El artículo 45 del Código Penal regula las costas procesales, estableciendo la obligación del condenado de pagar los gastos del proceso judicial.",
        "El artículo 35 del Código Penal establece las penas privativas de libertad, describiendo sus diferentes tipos y duraciones.",
        "La omisión del deber de socorro, según el Código Penal, se refiere a la falta de auxilio a una persona que se encuentra en situación de peligro y necesita ayuda inmediata.",
        "El artículo 31 del Código Penal regula la responsabilidad penal de las personas jurídicas, estableciendo que podrán ser responsables penalmente de los delitos cometidos en su nombre o por su cuenta.",
        "El artículo 57 del Código Penal establece la inhabilitación absoluta como una pena accesoria que implica la pérdida definitiva de todos los honores, empleos y cargos públicos.",
        "El artículo 62 del Código Penal define la prescripción de las penas, estableciendo los plazos específicos tras los cuales no se podrá ejecutar la pena impuesta.",
        "El delito de cohecho, según el Código Penal, consiste en la solicitud o aceptación de un soborno por parte de una autoridad o funcionario público.",
        "El artículo 138 del Código Penal regula el homicidio, estableciendo las penas para quien mate a otro sin concurrir las circunstancias de asesinato.",
        "El artículo 181 del Código Penal establece las penas y circunstancias agravantes para los delitos de abusos sexuales, diferenciándolos de las agresiones sexuales.",
        "El delito de falsedad documental, según el Código Penal, incluye la alteración, simulación o creación de documentos falsos con el fin de engañar.",
        "El artículo 237 del Código Penal regula el robo, estableciendo las penas para quien se apodere de cosas muebles ajenas con ánimo de lucro y empleando fuerza en las cosas para acceder o abandonar el lugar.",
        "El artículo 248 del Código Penal establece las penas para los delitos de estafa, definiendo esta como el uso de engaño con ánimo de lucro para causar un perjuicio patrimonial a otro.",
        "El delito de blanqueo de capitales, según el Código Penal, incluye el acto de adquirir, poseer, usar o convertir bienes que proceden de actividades delictivas.",
        "El artículo 301 del Código Penal regula el tráfico de drogas, estableciendo penas para la producción, venta, transporte o distribución de sustancias estupefacientes o psicotrópicas.",
        "El artículo 317 del Código Penal establece las penas para los delitos contra los derechos de los trabajadores, incluyendo la imposición de condiciones laborales que perjudiquen su salud o dignidad.",
        "La asociación ilícita, según el Código Penal, se refiere a la organización de tres o más personas con el fin de cometer delitos.",
        "El artículo 367 del Código Penal regula el contrabando, estableciendo las penas para quien introduzca o extraiga mercancías prohibidas o sometidas a restricciones sin cumplir con los requisitos legales.",
        "El artículo 391 del Código Penal establece las penas para la quiebra fraudulenta, definiéndola como la ocultación de bienes o el engaño a los acreedores en un proceso de insolvencia.",
        "El delito de acoso sexual, según el Código Penal, incluye cualquier comportamiento de naturaleza sexual que resulte ofensivo y que sea realizado de manera insistente y reiterada.",
        "El artículo 400 del Código Penal regula la insolvencia punible, estableciendo penas para quien, con el propósito de eludir sus obligaciones, se declare en insolvencia sin estar realmente en esa situación.",
        "El artículo 419 del Código Penal establece las penas para el tráfico de influencias, definiéndolo como el uso indebido de la posición de una persona para influir en la decisión de otra en el ejercicio de sus funciones públicas.",
        "El delito de terrorismo, según el Código Penal, incluye la comisión de actos violentos con la finalidad de subvertir el orden constitucional o alterar gravemente la paz pública.",
        "El artículo 510 del Código Penal regula los delitos de odio, estableciendo penas para quienes promuevan o inciten al odio, la violencia o la discriminación por motivos de raza, religión, ideología, orientación sexual u otros.",
        "El artículo 551 del Código Penal establece las penas para el atentado contra la autoridad, definiéndolo como el uso de violencia o intimidación contra funcionarios públicos en el ejercicio de sus funciones.",
        "El delito de injurias, según el Código Penal, incluye toda expresión que lesione la dignidad de otra persona, menoscabando su fama o atentando contra su propia estimación.",
        "El artículo 623 del Código Penal regula los delitos leves contra el patrimonio, estableciendo penas para aquellos que cometan hurtos, daños o usurpaciones de menor gravedad.",
        "El artículo 639 del Código Penal establece las penas para la usurpación de funciones públicas, definiéndola como el ejercicio de actos propios de una autoridad o funcionario público sin serlo.",
        "La omisión del deber de socorro, según el Código Penal, se refiere a la falta de auxilio a una persona que se encuentra en situación de peligro y necesita ayuda inmediata.",
        "El artículo 637 del Código Penal regula las penas para quienes alteren el orden público, participando en tumultos, desórdenes o disturbios.",
        "El artículo 641 del Código Penal establece las penas para la tenencia ilícita de armas, definiéndola como la posesión de armas de fuego o explosivos sin la correspondiente autorización.",
        "El delito de tráfico de órganos, según el Código Penal, incluye la extracción, comercialización o transporte ilegal de órganos humanos.",
        "El artículo 643 del Código Penal regula los delitos contra la salud pública, estableciendo penas para quienes pongan en peligro la salud de la población mediante la distribución de sustancias nocivas o peligrosas.",
        "El artículo 667 del Código Penal establece la responsabilidad penal del menor, definiendo las circunstancias y condiciones bajo las cuales los menores de edad pueden ser considerados responsables de sus actos delictivos."
    ],
    "answers": [
        {"answer_start": 0, "text": "Ley Orgánica 1/1982"},
        {"answer_start": 0, "text": "artículo 14"},
        {"answer_start": 0, "text": "artículo 15"},
        {"answer_start": 0, "text": "habeas corpus"},
        {"answer_start": 0, "text": "Ley Orgánica 3/2007"},
        {"answer_start": 0, "text": "artículo 47"},
        {"answer_start": 0, "text": "Ley de Enjuiciamiento Criminal"},
        {"answer_start": 0, "text": "artículo 20"},
        {"answer_start": 0, "text": "Tribunal Constitucional"},
        {"answer_start": 0, "text": "Ley de Propiedad Intelectual"},
        {"answer_start": 0, "text": "artículo 18"},
        {"answer_start": 0, "text": "Ley de Transparencia"},
        {"answer_start": 0, "text": "artículo 21"},
        {"answer_start": 0, "text": "derecho de asilo"},
        {"answer_start": 0, "text": "Ley de Protección de Datos Personales"},
        {"answer_start": 0, "text": "recurso de amparo"},
        {"answer_start": 0, "text": "artículo 27"},
        {"answer_start": 0, "text": "Ley General de la Seguridad Social"},
        {"answer_start": 0, "text": "Tribunal Supremo"},
        {"answer_start": 0, "text": "Ley de Arrendamientos Urbanos"}  ,    
        {"answer_start": 87, "text": "no este prevista como delito o falta por Ley anterior"},
        {"answer_start": 61, "text": "Juez o Tribunal competente"},
        {"answer_start": 48, "text": "peligrosidad criminal del sujeto"},
        {"answer_start": 104, "text": "delitos y faltas que se hallen penados por leyes especiales"},
        {"answer_start": 76, "text": "Son delitos o faltas las acciones y omisiones dolosas o imprudentes"},
        {"answer_start": 62, "text": "infracciones que la Ley castiga con pena grave"},
        {"answer_start": 157, "text": "Los menores de dieciocho años"},
        {"answer_start": 191, "text": "anomalia o alteracion psiquica"},
        {"answer_start": 307, "text": "da principio a la ejecucion del delito"},
        {"answer_start": 372, "text": "incita por medio de la imprenta, la radiodifusion o cualquier otro medio"},
        {"answer_start": 62, "text": "reeducación y reinserción social"},
        {"answer_start": 78, "text": "los menores de dieciocho años no serán responsables criminalmente"},
        {"answer_start": 63, "text": "extradición y el cumplimiento de penas impuestas por tribunales extranjeros"},
        {"answer_start": 67, "text": "no puede imponerse pena alguna sin previa ley que la establezca"},
        {"answer_start": 41, "text": "clases de penas, clasificándolas en graves, menos graves y leves"},
        {"answer_start": 31, "text": "multa como pena, incluyendo su cuantía y los criterios para su determinación"},
        {"answer_start": 71, "text": "actos inhumanos cometidos como parte de un ataque generalizado o sistemático"},
        {"answer_start": 52, "text": "prevención de la comisión de nuevos delitos"},
        {"answer_start": 42, "text": "suspensión de la ejecución de la pena en ciertos casos"},
        {"answer_start": 0, "text": "reincidencia"},
        {"answer_start": 48, "text": "concurso de delitos, estableciendo cómo se deben computar las penas"},
        {"answer_start": 31, "text": "responsabilidad civil derivada del delito, incluyendo la obligación de reparación"},
        {"answer_start": 19, "text": "legítima defensa, según el Código Penal, permite a una persona eximirse de responsabilidad criminal"},
        {"answer_start": 31, "text": "sustitución de las penas privativas de libertad por otras penas menos gravosas"},
        {"answer_start": 48, "text": "circunstancias atenuantes que pueden reducir la gravedad de la pena"},
        {"answer_start": 0, "text": "delito continuado"},
        {"answer_start": 42, "text": "libertad condicional a los penados que cumplan determinados requisitos"},
        {"answer_start": 67, "text": "se puede expulsar a un extranjero del territorio español como consecuencia de una condena penal"},
        {"answer_start": 48, "text": "cumplimiento sucesivo de penas"},
        {"answer_start": 41, "text": "causas de extinción de la responsabilidad criminal"},
        {"answer_start": 0, "text": "tentativa inacabada"},
        {"answer_start": 48, "text": "prescripción de delitos, estableciendo los plazos después de los cuales no puede perseguirse un delito"},
        {"answer_start": 43, "text": "prohibición de acercamiento a la víctima como medida de protección"},
        {"answer_start": 0, "text": "concurso medial"},
        {"answer_start": 48, "text": "reincidencia en delitos graves, estableciendo condiciones más severas"},
        {"answer_start": 41, "text": "medidas de seguridad que se pueden imponer a los delincuentes peligrosos"},
        {"answer_start": 31, "text": "peligrosidad criminal"},
        {"answer_start": 31, "text": "costas procesales, estableciendo la obligación del condenado de pagar los gastos"},
        {"answer_start": 41, "text": "penas privativas de libertad, describiendo sus diferentes tipos y duraciones"},
        {"answer_start": 31, "text": "omisión del deber de socorro, según el Código Penal, se refiere a la falta de auxilio"},
        {"answer_start": 53, "text": "delitos cometidos en su nombre o por su cuenta"},
        {"answer_start": 78, "text": "inhabilitación absoluta como una pena accesoria"},
        {"answer_start": 54, "text": "plazos específicos tras los cuales no se podrá ejecutar la pena"},
        {"answer_start": 42, "text": "solicitud o aceptación de un soborno por parte de una autoridad o funcionario público"},
        {"answer_start": 51, "text": "penas para quien mate a otro sin concurrir las circunstancias de asesinato"},
        {"answer_start": 51, "text": "penas y circunstancias agravantes para los delitos de abusos sexuales"},
        {"answer_start": 64, "text": "alteración, simulación o creación de documentos falsos con el fin de engañar"},
        {"answer_start": 47, "text": "penas para quien se apodere de cosas muebles ajenas con ánimo de lucro"},
        {"answer_start": 37, "text": "penas para los delitos de estafa, definiendo esta como el uso de engaño"},
        {"answer_start": 63, "text": "adquirir, poseer, usar o convertir bienes que proceden de actividades delictivas"},
        {"answer_start": 48, "text": "penas para la producción, venta, transporte o distribución de sustancias estupefacientes"},
        {"answer_start": 67, "text": "penas para los delitos contra los derechos de los trabajadores"},
        {"answer_start": 58, "text": "organización de tres o más personas con el fin de cometer delitos"},
        {"answer_start": 59, "text": "penas para quien introduzca o extraiga mercancías prohibidas"},
        {"answer_start": 45, "text": "penas para la quiebra fraudulenta, definiéndola como la ocultación de bienes"},
        {"answer_start": 35, "text": "comportamiento de naturaleza sexual que resulte ofensivo"},
        {"answer_start": 53, "text": "penas para quien se declare en insolvencia sin estar realmente en esa situación"},
        {"answer_start": 61, "text": "penas para el tráfico de influencias, definiéndolo como el uso indebido de la posición"},
        {"answer_start": 43, "text": "actos violentos con la finalidad de subvertir el orden constitucional"},
        {"answer_start": 53, "text": "penas para quienes promuevan o inciten al odio, la violencia o la discriminación"},
        {"answer_start": 51, "text": "penas para el atentado contra la autoridad, definiéndolo como el uso de violencia"},
        {"answer_start": 38, "text": "expresión que lesione la dignidad de otra persona"},
        {"answer_start": 43, "text": "penas para aquellos que cometan hurtos, daños o usurpaciones de menor gravedad"},
        {"answer_start": 45, "text": "penas para la usurpación de funciones públicas, definiéndola como el ejercicio de actos propios"},
        {"answer_start": 49, "text": "falta de auxilio a una persona que se encuentra en situación de peligro"},
        {"answer_start": 43, "text": "penas para quienes alteren el orden público, participando en tumultos"},
        {"answer_start": 45, "text": "penas para la tenencia ilícita de armas, definiéndola como la posesión de armas"},
        {"answer_start": 43, "text": "extracción, comercialización o transporte ilegal de órganos humanos"},
        {"answer_start": 57, "text": "penas para quienes pongan en peligro la salud de la población mediante la distribución de sustancias"},
        {"answer_start": 45, "text": "responsabilidad penal del menor, definiendo las circunstancias y condiciones"}
    ]
}

dataset = Dataset.from_dict(data)

## Data Pre-Processing

We create a function that ensures that the model can learn to identify the start and end positions of answers within the context, making it suitable for training a question-answering system.

# Train-Test Split for model Training

Split data into train and evaluation sets and tokenize.

In [ ]:
from utils import preprocess_function

train_dataset, eval_dataset = dataset.train_test_split(test_size=0.2).values()

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenized_train_dataset = train_dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True)
tokenized_eval_dataset = eval_dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True)

# Fine-Tuning Model

In [ ]:
# We will use BERT
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

# Training arguments and directory to save the trained model
training_args = TrainingArguments(
    output_dir="/notebooks/TFM_LAW_LLM/fine_tuning_trained_model/results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="/notebooks/TFM_LAW_LLM/fine_tuning_trained_model/logs", 
    logging_steps=100,     
)

We now train the model.

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset
)

# Train the model
trainer.train()

model.save_pretrained("/notebooks/TFM_LAW_LLM/fine_tuning_trained_model/results")
tokenizer.save_pretrained("/notebooks/TFM_LAW_LLM/fine_tuning_trained_model/results")

In [ ]:
# We load the tokenizer and the fine-tuned model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForQuestionAnswering.from_pretrained("/notebooks/TFM_LAW_LLM/fine_tuning_trained_model/results")  

# Question and Context

We create the question, and we provide a context so the model can find the answer in it.

In [ ]:
question = "¿Cual es la diferencia entre un homicidio y un asesinato?"
context = """
En el derecho penal español, tanto el homicidio como el asesinato son delitos que implican la muerte de una persona, pero se diferencian en sus características y gravedad. 
El homicidio, regulado en el artículo 138 del Código Penal, se define como causar la muerte a otra persona sin que concurran circunstancias agravantes. 
Por otro lado, el asesinato, tipificado en el artículo 139 del Código Penal, es una forma agravada de homicidio que incluye circunstancias específicas como la alevosía, el ensañamiento o la comisión del delito para facilitar otro delito o evitar ser descubierto.
Estas diferencias se reflejan en las penas aplicables: el homicidio conlleva una pena de prisión de 10 a 15 años, mientras que el asesinato se castiga con una pena de 15 a 25 años, pudiendo incluso llegar a la prisión permanente revisable en casos extremadamente graves.
"""

# Input Pipeline

In [ ]:
# Tokenize the input
inputs = tokenizer(
    question,
    context,
    max_length=512,
    truncation="only_second",
    padding="max_length",
    return_tensors="pt"
)

# Get the input IDs and attention mask
input_ids = inputs["input_ids"].to(model.device)
attention_mask = inputs["attention_mask"].to(model.device)

# Output of the Fine-Tuned model

In [ ]:
# Run the model
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

# We get the start and end positions 
start_scores = outputs.start_logits
end_scores = outputs.end_logits

# Once we have the start and end positions, we convert the scores to probabilities
start_prob = torch.softmax(start_scores, dim=1)
end_prob = torch.softmax(end_scores, dim=1)

# Now, we get the most probable start and end positions
start_index = torch.argmax(start_prob)
end_index = torch.argmax(end_prob)

Results:

In [ ]:
# We are now able gto extract the answer from the context
all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
answer_tokens = all_tokens[start_index:end_index + 1]
answer = tokenizer.convert_tokens_to_string(answer_tokens)

print(f"Question: {question}")
print(f"Answer: {answer}")